In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("mobiles.csv")
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


### Q1.

In [3]:
stat_mean = df["sales"].mean()
stat_std  = df["sales"].std()
stat_out  = stat_mean + 2 * stat_std
stat_out

146.55150129273218

In [4]:
df_q1 = df.loc[df["sales"] > stat_out, ].reset_index(drop = True)
df_q1.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79
1,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22


In [5]:
len(df_q1)

16

In [6]:
df_q1["idx"] = (df_q1["ROM"] / 32) + (df_q1["RAM"] / 2) + \
(df_q1["num_front_camera"] + df_q1["num_rear_camera"]) + \
(df_q1["battery_capacity"] / 1000)

In [7]:
round(df_q1["idx"].mean(), 2)

11.01

### Q2.

In [8]:
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":].copy()
df_q2.head(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
1,2815,4.5,244,57149,0.04,1.39
4,2815,4.6,745,69149,0.02,5.15


In [9]:
df_q2.corr().abs().round(2)["sales"][:-1].max()

0.95

In [10]:
df_q2.corr().replace(1, np.nan).max()

battery_capacity    0.257373
ratings             0.226075
num_of_ratings      0.949114
sales_price         0.151153
discount_percent    0.257373
sales               0.949114
dtype: float64

In [11]:
df_q2_corr = df_q2.corr()
df_q2_melt = df_q2_corr.reset_index().melt(id_vars = "index")
df_q2_melt = df_q2_melt.loc[df_q2_melt["index"] != df_q2_melt["variable"], ]
df_q2_melt.head()

,index,variable,value
1,ratings,battery_capacity,-0.424129
2,num_of_ratings,battery_capacity,0.034902
3,sales_price,battery_capacity,-0.503019
4,discount_percent,battery_capacity,0.257373
5,sales,battery_capacity,0.025680


In [12]:
df_q2_melt.pivot(index = "index", columns = "variable", values = "value")

variable,battery_capacity,discount_percent,num_of_ratings,ratings,sales,sales_price
index,,,,,,
battery_capacity,NaN,0.257373,0.034902,-0.424129,0.025680,-0.503019
discount_percent,0.257373,NaN,0.212442,-0.118578,0.223471,-0.098640
num_of_ratings,0.034902,0.212442,NaN,0.191655,0.949114,-0.260279
ratings,-0.424129,-0.118578,0.191655,NaN,0.226075,0.151153
sales,0.025680,0.223471,0.949114,0.226075,NaN,-0.247760
sales_price,-0.503019,-0.098640,-0.260279,0.151153,-0.247760,NaN


### Q3.

In [13]:
# df_q3_dum = pd.get_dummies(df, columns = ["screen_size"]) # 시험버전 
df_q3_dum = pd.get_dummies(df, dtype = "int") # Pandas 2.0.0 이상
df_q3_dum.head(1)

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,64,2,1,1,1800,4.5,38645,32999,0.17,127.52,0,0,0,0,1


In [14]:
df_q3_dum.shape

(430, 15)

In [15]:
df_q3_dum = df_q3_dum.set_index("sales").reset_index()
df_q3_dum.head(1)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,127.52,64,2,1,1,1800,4.5,38645,32999,0.17,0,0,0,0,1


In [16]:
df_train, df_test = train_test_split(df_q3_dum, train_size = 0.8, 
                                     random_state = 123)
len(df_train), len(df_test)

(344, 86)

In [17]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [18]:
arr_train_nor[:1, ]

array([[0.00394753, 0.04761905, 0.18181818, 0.33333333, 0.        ,
        0.42307692, 0.625     , 0.00396262, 0.02607261, 0.09302326,
        0.        , 0.        , 1.        , 0.        , 0.        ]])

In [19]:
df_train_nor = pd.DataFrame(arr_train_nor, columns = df_train.columns)
df_train_nor.head(1)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.003948,0.047619,0.181818,0.333333,0.0,0.423077,0.625,0.003963,0.026073,0.093023,0.0,0.0,1.0,0.0,0.0


In [21]:
ls_k = [3, 5, 7, 9, 11]

k = ls_k[0]

model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = arr_train_nor[:, 1:],
              y = arr_train_nor[:,  0])
pred = model_knn.predict(arr_test_nor[:, 1:])
mean_squared_error(y_true = arr_test_nor[:, 0], y_pred = pred) ** 0.5

0.08186677375964535

In [22]:
ls_k = [3, 5, 7, 9, 11]

ls_rmse = []
for k in ls_k:
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = arr_train_nor[:, 1:],
                  y = arr_train_nor[:,  0])
    pred = model_knn.predict(arr_test_nor[:, 1:])
    val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0], y_pred = pred) ** 0.5
    ls_rmse = ls_rmse + [val_rmse]

In [39]:
k

11

In [23]:
ls_rmse

[0.08186677375964535,
 0.09879109824384892,
 0.107669855645971,
 0.11232111394853059,
 0.1136902366621185]

In [24]:
ser_rmse = pd.Series(ls_rmse, index = ls_k)
ser_rmse

3     0.081867
5     0.098791
7     0.107670
9     0.112321
11    0.113690
dtype: float64

In [26]:
best_k = ser_rmse.idxmin()
best_k

3

### Q3. 추가 지시사항
다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가?  
※ 정규화 되지 않은 값으로 반올림하여 소수점 첫째 자리까지 출력하시오  
※ KNN 모델을 사용하며 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하시오.  
* ROM: 256
* RAM: 6
* num_rear_camera: 4
* num_front_camera: 1
* battery_capacity: 4000
* ratings: 4.3
* num_of_ratings: 25000
* sales_price: 85000
* discount_percent: 0.05
* screen_size: "Large"


In [35]:
# df_t1 = pd.DataFrame(dict(ROM = 256, ...))
df_t1 = df_test.head(1).reset_index(drop = True)
df_t1["RAM"] = 6
df_t1["num_rear_camera"] = 4
df_t1["battery_capacity"] = 4000
df_t1["ratings"] = 4.3
df_t1["num_of_ratings"] = 25000
df_t1["sales_price"] = 85000
df_t1["discount_percent"] = 0.05
df_t1["screen_size_Large"] = 1
df_t1["screen_size_Medium"] = 0
df_t1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,6,4,1,4000,4.3,25000,85000,0.05,1,0,0,0,0


In [36]:
MinMaxScaler().fit_transform(df_t1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [37]:
df_t1.drop(columns = "sales")

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,256,6,4,1,4000,4.3,25000,85000,0.05,1,0,0,0,0


In [38]:
# arr_t1_nor = model_nor.transform(df_t1.drop(columns = "sales"))
arr_t1_nor = model_nor.transform(df_t1)
arr_t1_nor

array([[0.0119438 , 0.49206349, 0.45454545, 1.        , 0.        ,
        0.42307692, 0.625     , 0.05308122, 0.51815842, 0.09302326,
        1.        , 0.        , 0.        , 0.        , 0.        ]])

In [40]:
model_knn_best = KNeighborsRegressor(n_neighbors = best_k)
model_knn_best.fit(X = arr_train_nor[:, 1:],
                   y = arr_train_nor[:,  0])
pred_t1 = model_knn_best.predict(arr_t1_nor[:, 1:])
pred_t1

array([0.00132259])

In [44]:
# model_nor.inverse_transform(pred_t1)
# model_nor.inverse_transform([pred_t1])
arr_t1_nor[0, 0] = pred_t1
arr_t1_inv = model_nor.inverse_transform(arr_t1_nor)
arr_t1_inv

array([[6.53333333e-01, 2.56000000e+02, 6.00000000e+00, 4.00000000e+00,
        1.00000000e+00, 4.00000000e+03, 4.30000000e+00, 2.50000000e+04,
        8.50000000e+04, 5.00000000e-02, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [45]:
df_t1_inv = pd.DataFrame(arr_t1_inv, columns = df_t1.columns)
df_t1_inv

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.653333,256.0,6.0,4.0,1.0,4000.0,4.3,25000.0,85000.0,0.05,1.0,0.0,0.0,0.0,0.0


In [46]:
df_t1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,6,4,1,4000,4.3,25000,85000,0.05,1,0,0,0,0


In [47]:
df_t1_inv["sales"]

0    0.653333
Name: sales, dtype: float64